In [1]:
import uindex as u
import sys

def p(*args):
    print(*args, flush=True, file=sys.stderr)

def print_stats(stats):
    for stat, val in sorted(stats.items()):
        p(f'{stat:<30}: {val:>13.2f}')

p('Reading..')
text = u.read()
num_queries = 1_000_000
_queries = []
_ql = 0
# k=0 to disable minimizer space
# Returns: (space, total construction time, query time)
def run(ql, l, k, remap, store_ms_seq, compress, cacheline_ef=False):
    p('Building..')
    if k == 0:
        ui = u.build_plain(text)
    else:
        ui = u.build_minimized(text, k, l, remap, store_ms_seq, compress, cacheline_ef)
    
    global _ql, _queries
    if ql != _ql or not _queries:
        p('Regenerating queries..')
        _queries = u.gen_queries(ui, ql, num_queries)
        _ql = ql
        
    p('Benching..')
    bench_time = u.bench(ui, _queries)
    p(f'Benching: {bench_time}s')

    p('Stats')
    stats = u.stats(ui)

    build_time = stats['Sketch'] + stats['Build']

    stats['query_length'] = ql
    stats['l'] = l
    stats['k'] = k
    stats['remap'] = remap
    stats['store_ms_seq'] = store_ms_seq
    stats['compress'] = compress
    stats['build_time'] = build_time
    stats['query_time'] = bench_time
    print_stats(stats)

    return stats
    


Reading..
2024-09-30T16:34:16.466140Z TRACE uindex: Read human genome: 25 chromosomes of total length 3117Mbp and size 779MB
2024-09-30T16:34:16.471011Z TRACE uindex::utils:  Reading                       : 2.565s


In [4]:
ql = 256
#lks = [(0, 0), (16, 4), (64, 8), (32, 8), (128, 16), (64, 16), (256, 32), (128, 32), (64, 32),]
lks = [(16, 4), (32, 8), (128, 16), (64, 16), (128, 28)]
results = []
for (l, k) in lks:
    if k == 0:
        results.append(run(ql, l, k, False, False))
    else:
        for remap in [False]:
            for compress in [True]:
                for store_ms_seq in [True, False]:
                    results.append(run(ql, l, k, remap, store_ms_seq, compress))


query_length                  :        256.00


query_matches                 :    1808654.00


query_misaligned_ms_pos       :          0.00


query_mismatches              :          0.00


query_out_of_bounds           :          0.00


...

sequence length               :  126340680.00


sequence_length               :  248387328.00


sketch_size_MB                :         12.06


total_size_MB                 :        201.57


2024-09-24T01:27:17.490785Z  INFO uindex: Query stats: QueryStats {
    too_short: 0,
    unknown_minimizer: 0,
    misaligned_ms_pos: 0,
    out_of_bounds: 0,
    mismatches: 590457,
    matches: 1808654,
}


In [3]:
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 240)
pd.set_option('display.precision', 2)
pd.options.display.float_format = "{:,.1f}".format

rs = results[:]
rs = pd.DataFrame(rs)
rs = rs[(rs.compress==True) | (rs.k==0)]
#rs = rs[rs.remap==False]
# display(rs)
rs['query_mismatch_rel'] = rs['query_mismatches'] / rs['query_matches']
rs['query_misaligned_rel'] = rs['query_misaligned_ms_pos'] / rs['query_matches']
print('Sequence length:', rs['seq_size_MB'].unique(), 'Mbp')
print('Query length: ', rs['query_length'].unique())
sub = rs[['l', 'k', 'remap', 'seq_size_MB', 'total_size_MB', 'sketch_size_MB', 'ms_seq_size_MB', 'sa_size_MB', 'num_minimizers', 'build_time', 'Sketch', 'Build', 'query_time', 'kmer_width_bits', 'query_mismatch_rel']]
sub


Sequence length: [779.32305908] Mbp
Query length:  [256]


,l,k,remap,seq_size_MB,total_size_MB,sketch_size_MB,ms_seq_size_MB,sa_size_MB,num_minimizers,build_time,Sketch,Build,query_time,kmer_width_bits,query_mismatch_rel
0,16,4,False,779.3,"2,492.3",271.8,444.1,"1,776.4","444,107,680.0",63.6,17.1,46.5,10.5,8.0,0.8
1,16,4,False,779.3,"2,048.2",271.8,0.0,"1,776.4","444,107,680.0",62.8,16.9,45.9,17.2,8.0,0.8
2,32,8,False,779.3,"1,609.2",172.6,478.8,957.7,"239,421,392.0",52.7,13.1,39.6,12.2,16.0,2.0
3,32,8,False,779.3,"1,130.3",172.6,0.0,957.7,"239,421,392.0",52.5,13.0,39.6,17.1,16.0,2.0
4,128,16,False,779.3,490.8,54.1,218.4,218.4,"54,590,160.0",25.5,9.8,15.7,20.9,32.0,6.4
5,128,16,False,779.3,272.4,54.1,0.0,218.4,"54,590,160.0",25.1,9.5,15.6,23.7,32.0,6.4
6,64,16,False,779.3,"1,095.4",104.0,495.7,495.7,"123,922,656.0",49.8,11.2,38.6,9.9,32.0,1.1
7,64,16,False,779.3,599.7,104.0,0.0,495.7,"123,922,656.0",49.9,10.9,39.0,13.3,32.0,1.1
8,128,28,False,779.3,747.8,60.2,437.6,250.1,"62,514,536.0",43.0,9.4,33.6,12.9,56.0,2.8
9,128,28,False,779.3,310.2,60.2,0.0,250.1,"62,514,536.0",43.1,9.6,33.6,16.1,56.0,2.8


In [ ]:
sub.loc[[0,2,6,8,10,14,16]].sort_values('k')
